# Prep CESM-DPLE experiment data for use in DPLE processing notebook
### • eliminates depth dimension

In [1]:
import os
from glob import glob

import matplotlib.colors as colors

import numpy as np
import xarray as xr
import pop_tools
import utils
import pop_tools
import warnings
warnings.filterwarnings('ignore')

## Spin up a dask cluster

In [2]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='50GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=50GB',
        project='NCGD0011',
        walltime='06:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
    })
    client = Client(cluster)
    return cluster, client

In [3]:
cluster, client = get_ClusterClient()
cluster.scale(12) 

In [4]:
client

Client Scheduler: tcp://10.12.205.21:32904 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kristenk/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
#cluster.close()

In [6]:
grid = pop_tools.get_grid('POP_gx1v6')

## cycle through start years (1954 - 2017; n=64) and ensemble members (n=40) and do preprocessing

In [7]:
#user set/ cycle vars

for var in ['zooC']: #, 'diatC', 'zoo_loss', 'spC', 'zooC', 'TEMP']:
    
    for styear in np.arange(1954,2018,1):

        for member in np.arange(1,41,1):

            print('start year=', styear, 'member=', member)

            #first get this particular file
            ds = xr.open_dataset(
            utils.file_in(var,styear,member), 
            decode_coords=False, 
            decode_times=True,
            chunks={'time': 12}).persist()

            #tidy up the dataset
            coords = {'x':'TLONG','y':'TLAT'}
            keepthese=['z_t','z_t_150m','time_bound','TAREA','REGION_MASK','time','UAREA'] + [var]
            keep_vars = keepthese +list(coords.values())+['dz','KMT']
            ds = ds.drop([v for v in ds.variables if v not in keep_vars])

            #prepare output file -- it needs certain variables add to work with the DPLE code
            USER = os.environ['USER']
            dout = f'/glade/scratch/{USER}/DPLE-tmpfiles3/'
            os.makedirs(dout, exist_ok=True)
            file_out = utils.file_out(dout,var,styear,member)
            dso = ds[['TLONG', 'TLAT', 'KMT', 'TAREA','time','time_bound','dz']] 

            #reduce depth dimension
            if (var=='TEMP'):
                #TEMP top 150m mean
                dso['TEMP_150m'] = utils.field_150m_mean(ds[var]).compute()
                dso['TEMP_150m'].attrs['coordinates'] = "TLONG TLAT time"
            elif ('z_t_150m' in ds[var].dims):
                dso[f'{var}_150m'] = utils.field_150m_zint(ds[var], ds.dz).compute()
                dso[f'{var}_150m'].attrs['coordinates'] = "TLONG TLAT time"
                dso[f'{var}_150m'] = dso[f'{var}_150m'].where(grid.REGION_MASK>0)
            #write out the file
            dso.to_netcdf(file_out, mode='w')

start year= 1954 member= 1
start year= 1954 member= 2
start year= 1954 member= 3
start year= 1954 member= 4
start year= 1954 member= 5
start year= 1954 member= 6
start year= 1954 member= 7
start year= 1954 member= 8
start year= 1954 member= 9
start year= 1954 member= 10
start year= 1954 member= 11
start year= 1954 member= 12
start year= 1954 member= 13
start year= 1954 member= 14
start year= 1954 member= 15
start year= 1954 member= 16
start year= 1954 member= 17


distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", line 493, in handle_comm
    result = handler(comm, **msg)
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in heartbeat_worker
    self.tasks[key]: duration for key, duration in executing.items()
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in <dictcomp>
    self.tasks[key]: duration for key, duration in executing.items()
KeyError: "('open_dataset-461e5d222c3c5c4e3d773bbc1b7ff637HTE-80964dfa8cf45f53410baecb8ea55c39', 0, 0)"
distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", lin

start year= 1954 member= 18
start year= 1954 member= 19
start year= 1954 member= 20
start year= 1954 member= 21


distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", line 493, in handle_comm
    result = handler(comm, **msg)
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in heartbeat_worker
    self.tasks[key]: duration for key, duration in executing.items()
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in <dictcomp>
    self.tasks[key]: duration for key, duration in executing.items()
KeyError: "('open_dataset-9a4adf2297e27ba0d94bd1234154ae9aDXT-5ce6cff69a337d7b5ba8179d87e58344', 0, 0)"


start year= 1954 member= 22
start year= 1954 member= 23
start year= 1954 member= 24
start year= 1954 member= 25
start year= 1954 member= 26
start year= 1954 member= 27
start year= 1954 member= 28
start year= 1954 member= 29
start year= 1954 member= 30
start year= 1954 member= 31
start year= 1954 member= 32
start year= 1954 member= 33
start year= 1954 member= 34
start year= 1954 member= 35
start year= 1954 member= 36
start year= 1954 member= 37
start year= 1954 member= 38
start year= 1954 member= 39
start year= 1954 member= 40
start year= 1955 member= 1


...

start year= 2016 member= 29
start year= 2016 member= 30
start year= 2016 member= 31
start year= 2016 member= 32
start year= 2016 member= 33
start year= 2016 member= 34
start year= 2016 member= 35
start year= 2016 member= 36
start year= 2016 member= 37
start year= 2016 member= 38


distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", line 493, in handle_comm
    result = handler(comm, **msg)
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in heartbeat_worker
    self.tasks[key]: duration for key, duration in executing.items()
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in <dictcomp>
    self.tasks[key]: duration for key, duration in executing.items()
KeyError: "('open_dataset-dbc89270dfa975ff6fa10cf8d53e8f0dDXT-e5738eaf98b7da997890771d2f4ca217', 0, 0)"


start year= 2016 member= 39
start year= 2016 member= 40
start year= 2017 member= 1
start year= 2017 member= 2
start year= 2017 member= 3
start year= 2017 member= 4
start year= 2017 member= 5
start year= 2017 member= 6
start year= 2017 member= 7
start year= 2017 member= 8
start year= 2017 member= 9
start year= 2017 member= 10
start year= 2017 member= 11
start year= 2017 member= 12
start year= 2017 member= 13
start year= 2017 member= 14
start year= 2017 member= 15
start year= 2017 member= 16
start year= 2017 member= 17
start year= 2017 member= 18
start year= 2017 member= 19
start year= 2017 member= 20
start year= 2017 member= 21
start year= 2017 member= 22
start year= 2017 member= 23
start year= 2017 member= 24
start year= 2017 member= 25
start year= 2017 member= 26
start year= 2017 member= 27
start year= 2017 member= 28
start year= 2017 member= 29
start year= 2017 member= 30
start year= 2017 member= 31
start year= 2017 member= 32
start year= 2017 member= 33
start year= 2017 member= 34
s

distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", line 493, in handle_comm
    result = handler(comm, **msg)
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in heartbeat_worker
    self.tasks[key]: duration for key, duration in executing.items()
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/scheduler.py", line 2197, in <dictcomp>
    self.tasks[key]: duration for key, duration in executing.items()
KeyError: "('open_dataset-f76d7056886d1e8a2a8f67f06a7acccaDYT-69a218326ffde90f0374eadbb61ff0ca', 0, 0)"
distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "/glade/work/kristenk/miniconda/envs/analysis4/lib/python3.7/site-packages/distributed/core.py", lin

start year= 2017 member= 39
start year= 2017 member= 40


In [ ]:
cluster.close()